In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash  # Enables running Dash applications in Jupyter notebooks

# Import necessary modules for dashboard components
import dash_leaflet as dl  # For geolocation map visualization
from dash import dcc, html, dash_table  # Dash components for interactivity and layout
from dash.dependencies import Input, Output  # Defines inputs and outputs for interactivity
import plotly.express as px  # Library for creating charts and graphs
import base64  # Used for encoding the Grazioso Salvare logo
import pandas as pd  # For data manipulation and querying
import socket  # For automatic port selection

# Import the CRUD module for MongoDB operations
from mongo_crud import AnimalShelter  # Custom Python module for MongoDB CRUD functionality

###########################
# Data Manipulation / Model
###########################

# MongoDB Authentication - Connect to database using valid credentials
username = "aacuser"  # Username for MongoDB
password = "kakarot"  # Password for MongoDB
shelter = AnimalShelter()  # Instantiate the AnimalShelter class with credentials

# Fetch all data from MongoDB into a Pandas DataFrame
df = pd.DataFrame.from_records(shelter.read({}))  # Perform a "retrieve all" query
df.drop(columns=['_id'], inplace=True)  # Drop '_id' column to avoid rendering issues with Dash components

#########################
# Dashboard Layout / View
#########################

# Initialize the Jupyter Dash application
app = JupyterDash(__name__)

# Encode Grazioso Salvare logo for embedding in the app
image_filename = 'grazioso_logo.png'  # Path to the logo image file
encoded_image = base64.b64encode(open(image_filename, 'rb').read())  # Encode image in base64 format

# Define the dashboard layout
app.layout = html.Div([

    # Title section
    html.Center(html.B(html.H1('CS-340 Dashboard'))),  # Display the dashboard title

    # Display Grazioso Salvare logo (encoded as base64) for branding
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '150px'})),

    # Unique identifier to credit the developer
    html.Center(html.H3("Developed by Miguel Hilario")),  # Unique Identifier

    html.Hr(),  # Horizontal line for separation

    # Interactive filter options for rescue types
    html.Div([
        html.Label("Filter Rescue Type"),  # Label for filter section
        dcc.RadioItems(  # Create radio buttons for filtering rescue types
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},  # Option for water rescue
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},  # Option for mountain/wilderness rescue
                {'label': 'Disaster/Tracking Rescue', 'value': 'disaster'},  # Option for disaster/tracking rescue
                {'label': 'Reset Filters', 'value': 'reset'}  # Reset filters to unfiltered view
            ],
            value='reset',  # Default value to show all data
            inline=True  # Display radio buttons horizontally
        )
    ]),
    html.Hr(),  # Horizontal line for separation

    # DataTable for displaying animal data
    dash_table.DataTable(
        id='datatable-id',  # Unique ID for identifying the DataTable in callbacks
        columns=[
            {"name": col, "id": col, "deletable": False, "selectable": True} for col in df.columns
        ],  # Dynamically generate column headers from DataFrame
        data=df.to_dict('records'),  # Populate DataTable with initial data
        style_table={'overflowX': 'auto'},  # Enable horizontal scrolling for wide tables
        style_cell={'textAlign': 'left', 'padding': '5px'},  # Align cell text to the left with padding
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},  # Style headers for emphasis
        page_size=10,  # Display 10 rows per page
        sort_action='native',  # Enable sorting by clicking column headers
        row_selectable='single',  # Allow selection of a single row
        selected_rows=[0],  # Default to selecting the first row
    ),
    html.Br(),  # Add space between components
    html.Hr(),  # Horizontal line for separation

    # Section for charts and geolocation map
    html.Div(className='row',
             style={'display': 'flex'},  # Ensure side-by-side display of charts and map
             children=[
                 html.Div(
                     id='graph-id',  # ID for the pie chart
                     className='col s12 m6',  # Styling for half-width
                 ),
                 html.Div(
                     id='map-id',  # ID for the geolocation map
                     className='col s12 m6',  # Styling for half-width
                 )
             ]),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update DataTable based on filter selection
@app.callback(
    Output('datatable-id', 'data'),  # Update data in the DataTable
    [Input('filter-type', 'value')]  # Triggered by changes in radio button selection
)
def update_dashboard(filter_type):
    # Define MongoDB queries based on selected rescue type
    query = {}
    if filter_type == 'water':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever"]}}
    elif filter_type == 'mountain':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute"]}}
    elif filter_type == 'disaster':
        query = {"breed": {"$in": ["Beagle", "Bloodhound"]}}
    elif filter_type == 'reset':  # Reset to display all data
        query = {}

    # Fetch filtered data from MongoDB and return it to the DataTable
    filtered_data = pd.DataFrame.from_records(shelter.read(query))
    filtered_data.drop(columns=['_id'], inplace=True)  # Remove '_id' column
    return filtered_data.to_dict('records')  # Return filtered data as dictionary

# Update Pie Chart based on filtered data
@app.callback(
    Output('graph-id', 'children'),  # Update children of the pie chart section
    [Input('datatable-id', 'derived_virtual_data')]  # Triggered by changes in DataTable data
)
def update_graphs(viewData):
    if not viewData:  # Handle case where no data is available
        return html.Div("No data available for graph.")
    dff = pd.DataFrame.from_dict(viewData)  # Convert data to DataFrame
    return [
        dcc.Graph(
            figure=px.pie(
                dff,
                names='breed',  # Display breed distribution
                title='Rescue Breed Distribution'  # Title for the pie chart
            )
        )
    ]

# Update Geolocation Map based on DataTable row selection
@app.callback(
    Output('map-id', 'children'),  # Update children of the map section
    [Input('datatable-id', 'derived_virtual_data'),  # Triggered by changes in DataTable data
     Input('datatable-id', 'selected_rows')]  # Triggered by row selection
)
def update_map(viewData, index):
    if not viewData or not index:  # Handle case where no data or row is selected
        return html.Div("No data available for mapping.")
    dff = pd.DataFrame.from_dict(viewData)  # Convert data to DataFrame
    row = index[0] if index else 0  # Default to first row if no row is selected
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),  # Base map layer
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),  # Display breed as tooltip
                dl.Popup([
                    html.H1("Animal Name"),  # Popup title
                    html.P(dff.iloc[row, 9])  # Popup content displaying name
                ])
            ])
        ])
    ]

# Automatically find a free port because I am tired of that error
def find_free_port():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('', 0))  # Bind to any available port
    port = sock.getsockname()[1]  # Get the free port number
    sock.close()
    return port

# Run the app on a free port
free_port = find_free_port()
app.run_server(debug=True, port=free_port)